# Encoding


ASCII es un estándar de codificación de 7 bits que permite la representación de texto utilizando números enteros del 0 al 127.

Usando la siguiente matriz de enteros, convierta los números a sus caracteres ASCII correspondientes para obtener una bandera.

```
[99, 114, 121, 112, 116, 111, 123, 65, 83, 67, 73, 73, 95, 112, 114, 49, 110, 116, 52, 98, 108, 51, 125]
```

> En Python, la función `chr()` se puede utilizar para convertir un número ordinal ASCII en un carácter (la función `ord()` hace lo contrario).




In [ ]:
array = [99, 114, 121, 112, 116, 111, 123, 65, 83, 67, 73, 73, 95, 112, 114, 49, 110, 116, 52, 98, 108, 51, 125]

print("".join(chr(c) for c in array))

crypto{ASCII_pr1nt4bl3}


# Hex

Cuando ciframos algo, el texto cifrado resultante suele tener bytes que no son caracteres ASCII imprimibles. Si queremos compartir nuestros datos cifrados, es común codificarlos en algo más fácil de usar y portátil en diferentes sistemas.

El hexadecimal se puede utilizar de tal manera para representar cadenas ASCII.
Primero, cada letra se convierte en un número ordinal según la tabla ASCII (como en el desafío anterior). Luego, los números decimales se convierten a números de base 16, también conocidos como hexadecimales. Los números se pueden combinar entre sí, en una larga cadena hexadecimal.

A continuación se incluye una bandera codificada como una cadena hexadecimal.
Decodifica esto nuevamente en bytes para obtener la bandera.

```
63727970746f7b596f755f77696c6c5f62655f776f726b696e675f776974685f6865785f737472696e67735f615f6c6f747d
```

> En Python, la función `bytes.fromhex()` se puede utilizar para convertir hexadecimal a bytes. Se puede invocar el método de instancia `.hex()` en cadenas de bytes para obtener la representación hexadecimal.

Recursos:
  - [ASCII table](https://www.rapidtables.com/code/text/ascii-table.html)
  - [Wikipedia: Hexadecimal (Español)](https://es.wikipedia.org/wiki/Sistema_hexadecimal)

In [ ]:
hex = "63727970746f7b596f755f77696c6c5f62655f776f726b696e675f776974685f6865785f737472696e67735f615f6c6f747d"
print(bytes.fromhex(hex))

# Base64

Otro esquema de codificación común es Base64, que nos permite representar datos binarios como una cadena ASCII utilizando un alfabeto de 64 caracteres. Un carácter de una cadena Base64 codifica 6 dígitos binarios (bits), por lo que 4 caracteres de Base64 codifican tres bytes de 8 bits.

Base64 se usa más comúnmente en línea, por lo que los datos binarios, como imágenes, se pueden incluir fácilmente en archivos HTML o CSS.

Tome la siguiente cadena hexadecimal, decodifíquela en bytes y luego codifíquela en Base64.

```
72bca9b68fc16ac7beeb8f849dca1d8a783e8acf9679bf9269f7bf
```

> En Python, después de importar el módulo base64 con import base64, puede usar la función base64.b64encode(). Recuerde decodificar el hexadecimal primero como lo indica la descripción del desafío.



In [ ]:
import base64 as base64
hex = "72bca9b68fc16ac7beeb8f849dca1d8a783e8acf9679bf9269f7bf"
print(base64.b64encode(bytes.fromhex(hex)))